In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-1.5B-instruct", trust_remote_code=True)
# In case you want to reduce the maximum length:
model.max_seq_length = 8192

# check vector dimensions
print(model.get_sentence_embedding_dimension())

C:\Users\skibi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.20s/it]


1536


In [2]:
import re


def clean_title(title):
    # Extract the authority and the rest of the title
    match = re.match(r'^(?:Rozporządzenie|Obwieszczenie)\s+(.*?)\s+z\s+dnia.*?(?:w sprawie|zmieniające)', title, re.IGNORECASE)
    authority = match.group(1) if match else ""
    
    # Remove the document type, date, and "w sprawie" phrases
    title = re.sub(r'^.*?(?:z dnia \d+\s+\w+\s+\d{4}\s*r\.\s*)', '', title)
    title = re.sub(r'(?:zmieniające\s+rozporządzenie\s+)?w\s+sprawie\s+', '', title)
    title = title.replace("Rzeczypospolitej Polskiej ogłoszenia jednolitego tekstu ustawy", "")
    # Combine authority with cleaned title
    cleaned_title = f"{authority} {title}".strip()
    
    # remove this type of things with (ALPHANUMERICBIGCASENOSPACES)
    # (PLH120079)
    cleaned_title = re.sub(r'\(\w+\d+\)', '', cleaned_title)
    # Remove extra whitespace
    cleaned_title = re.sub(r'\s+', ' ', cleaned_title).strip()
    patterns_to_remove = [
        r"Ministra",
        r"Marszałka Sejmu Rzeczypospolitej Polskiej",
        r"Prezesa Rady Ministrów",
        r"Rady Ministrów",
        r"ogłoszenia jednolitego tekstu",
        r"zmieniające rozporządzenie",
    ]
    for pattern in patterns_to_remove:
        text = re.sub(pattern, "", cleaned_title)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and extra whitespace
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()


    return cleaned_title
text = "Rozporządzenie Ministra Klimatu i Środowiska z dnia 9 grudnia 2022 r. w sprawie funkcjonowania Bazy danych o produktach i opakowaniach oraz o gospodarce odpadami"
clean_title(text)

'Ministra Klimatu i Środowiska funkcjonowania Bazy danych o produktach i opakowaniach oraz o gospodarce odpadami'

In [5]:
# load documents from `acts.csv` `
import pandas as pd

acts = pd.read_csv("acts.csv", nrows=20)
acts['title'] = acts['title'].apply(clean_title)

# to csv
acts.to_csv("acts_cleaned.csv", index=False)


# there is only title column in the csv file
documents = acts["title"].tolist()
print(len(documents))
print(documents[:5])

20
['Ministra Klimatu i Środowiska funkcjonowania Bazy danych o produktach i opakowaniach oraz o gospodarce odpadami', 'Ministra Finansów adnotacji zbiegu egzekucji oraz dokonywania doręczeń przy wykorzystaniu systemu teleinformatycznego albo z użyciem środków komunikacji elektronicznej pomiędzy organami egzekucyjnymi oraz pomiędzy organem egzekucyjnym a komornikiem sądowym', 'Ministra Edukacji i Nauki warunków wynagradzania egzaminatorów za udział w przeprowadzaniu egzaminów oraz nauczycieli akademickich za udział w przeprowadzaniu części ustnej egzaminu maturalnego', 'Ministra Rolnictwa i Rozwoju Wsi wzoru paszportu bydła', 'Ministra Infrastruktury warunków ustalania oraz sposobu dokonywania zwrotu kosztów używania do celów służbowych samochodów osobowych, motocykli i motorowerów niebędących własnością pracodawcy']


In [6]:
# add embeddings to documents
embeddings = model.encode(documents, show_progress_bar=True)
print(embeddings.shape)

Batches:   0%|          | 0/1 [00:41<?, ?it/s]


KeyboardInterrupt: 

In [49]:
# test similarity
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Ensure the model is set to use the CPU
device = torch.device('cpu')
phrase = "Segregacja odpadów"
phrase_embedding = model.encode(phrase)
similarities = cosine_similarity([phrase_embedding], embeddings)

# Get the indices of the top 5 most similar documents
top_5_indices = similarities[0].argsort()[-5:][::-1]

# Print the top 5 most similar documents and their similarity scores
for index in top_5_indices:
    print(documents[index])
    print(similarities[0][index])

Ministra Klimatu i Środowiska mechaniczno-biologicznego przetwarzania niesegregowanych (zmieszanych) odpadów komunalnych
0.58494055
Ministra Klimatu i Środowiska specjalnego obszaru ochrony siedlisk Tylmanowa
0.42642993
Ministra Klimatu i Środowiska specjalnego obszaru ochrony siedlisk Lasy Bierzwnickie
0.4260394
Ministra Klimatu i Środowiska specjalnego obszaru ochrony siedlisk Karsibórz Świdwiński
0.4242239
Ministra Klimatu i Środowiska specjalnego obszaru ochrony siedlisk Jaskinia Oblica
0.4231249
